# Image Similarity
- Skimage (SSIM) - https://scikit-image.org/docs/stable/api/skimage.metrics.html#skimage.metrics.structural_similarity
- PIQ (DISTS) - https://github.com/photosynthesis-team/piq
- SIFT - https://github.com/adumrewal/SIFTImageSimilarity

For finding similar or duplicate photos  we need to extract a metric (similarity score) from a pair of images and then decide to what extent we consider images similar based on this number. There are two ways to get the similarity score. First one is image-based, where we compute the distance between pair of images after resizing them. Second one is distribution-based, where we extract features from images and compute the distance between pair of feature distributions.

For testing we created small folder with image and its various augmentations.

In [29]:
import time, os
image_path = '/home/lukas/Bakalářka/photo_culling/images/testing'
img_list = []  # list of image file names to process
for path in os.scandir(image_path):
    if path.is_file():
        if path.name.endswith(".jpg"):
            img_list += [path.name]

 We tested two image-based metrics. First one is a metric called Structural Similarity (SSIM) invented in year 2004. We used implementation from skimage library.
 (see https://scikit-image.org/docs/stable/api/skimage.metrics.html#skimage.metrics.structural_similarity)
For both image-based metrics we firstly need to reshape the images to same resolution to be able to compare the similarity. We consider two images similar if the similarity score is above 0.7.

In [86]:
# need to resize images to match
def imread_reshape(path,lst):
    shape = [224,224]
    images = None
    num = 0
    for image in lst:
        temp = torch.tensor(io.imread(os.path.join(path, image)))/255.
        if temp is not None:
            num+=1
            temp = skimage.transform.resize_local_mean(temp, output_shape=shape)
            temp = np.expand_dims(temp, 2)
            temp = np.rollaxis(temp, 2, 0).astype('f2')
            temp = np.rollaxis(temp, 3, 1).astype('f2')
            if images is None:
                images = temp
            else:
                images = np.append(images,temp,0).astype('f2')
    return images, num

tic = time.perf_counter()
images, img_num = imread_reshape(image_path,img_list)
toc = time.perf_counter()
print(f"TIME to process images - {toc-tic:0.2f} s")

TIME to process images - 2.30 s


In [111]:
import skimage, torch
import numpy as np
from skimage import  io

sim_list = [[0 for i in range(img_num)] for j in range(img_num)]
nbrs = 5
tic = time.perf_counter()
for i in range(img_num):
    x = skimage.color.rgb2gray(images[i],channel_axis=0)
    for j in range(img_num):
        if i<j:
            continue
        y = skimage.color.rgb2gray(images[j],channel_axis=0)
        sim_list[i][j] = skimage.metrics.structural_similarity(x, y, win_size=3, data_range= 1,sigma=1.5,use_sample_covariance=False,gaussian_weights=True)
        sim_list[j][i] = sim_list[i][j]
toc = time.perf_counter()
print(f"TIME to get all similar - {toc-tic:0.2f} s")
for i, line in enumerate(sim_list):
    line = np.round(line,decimals=3)
    print(np.trim_zeros(line),img_list[i],)
        

TIME to get all similar - 0.15 s
[1.    0.986 0.481 0.111 0.964 0.77 ] clear.jpg
[0.986 1.    0.488 0.128 0.955 0.755] GaussBlur.jpg
[0.481 0.488 1.    0.249 0.476 0.426] rotated.jpg
[0.111 0.128 0.249 1.    0.09  0.113] inverted.jpg
[0.964 0.955 0.476 0.09  1.    0.737] hue_shift.jpg
[0.77  0.755 0.426 0.113 0.737 1.   ] contrast.jpg


Resulting 2D array shows us similarity score between each pair of images. The diagonal of ones means that the images are duplicate which is obvious since we compare image to itself. Numbers that are closer to 1 mean that the images are more similar. We can see from the results that blurring the image or shifting the color hue doesn't result in significant score dropping. Changing the contrast causes a drop that might is significant enough that for some cases the similarity might not be detected. Rotating and inverting the image causes that the similarity is not detected. The big positive is extremely fast processing of the results.

The second image-based metric that we tested is called Deep Image Structure and Texture Similarity (DISTS) from the year 2020. We chose to use implementation from PyTorch Image Quality library made by Photosynthesis Team. This metric implements deep learning into the computing and as result is more accurate with significantly longer process times.
(see https://github.com/photosynthesis-team/piq)

In [112]:
import piq
import warnings

sim_list = [[0 for i in range(img_num)] for j in range(img_num)]
nbrs = 5
tic = time.perf_counter()
for i in range(img_num):
    x = torch.Tensor(images[i][None,...])
    for j in range(img_num):
        if i<j:
            continue
        y = torch.Tensor(images[j][None,...])
        temp = piq.DISTS(reduction='none')(x,y)
        sim_list[i][j] = 1 - temp.item()
        sim_list[j][i] = sim_list[i][j]
toc = time.perf_counter()
print(f"TIME to get all similar - {toc-tic:0.2f} s")
for i, line in enumerate(sim_list):
    line = np.round(line,decimals=3)
    print(np.trim_zeros(line),img_list[i],)


TIME to get all similar - 37.62 s
[1.    0.959 0.817 0.573 0.8   0.793] clear.jpg
[0.959 1.    0.795 0.558 0.782 0.771] GaussBlur.jpg
[0.817 0.795 1.    0.609 0.72  0.751] rotated.jpg
[0.573 0.558 0.609 1.    0.637 0.564] inverted.jpg
[0.8   0.782 0.72  0.637 1.    0.764] hue_shift.jpg
[0.793 0.771 0.751 0.564 0.764 1.   ] contrast.jpg


Results are similar to the ones from previous metric. The main difference is that rotated image is scored much higher than in previous test which is desirable as it detects rotated images as similar. Score for inverted images is still low as it would escape the similarity detection. Overall this algorithm has better behaviour for our usage but comes at cost of significantly higher processing time.

To avoid problems with rotated, inverted and scaled images escaping our detection we will have to test out algorithm that is distribution-based. In our case we chose algorithm called Scale Invariant Feature Transform (SIFT) from year 2004. Our implementation is inspired by code made by Amol Dumrewal.
(see https://github.com/adumrewal/SIFTImageSimilarity)

For this approach we consider images similar if the similarity score is higher than 0.1.

In [125]:
import cv2

def compute_SIFT(image):
    return sift.detectAndCompute(image, None)

def image_resize(image):
    max_d = 1024
    height,width,channel = image.shape
    aspect_ratio = width/height
    if aspect_ratio < 1:
        new_size = (int(max_d*aspect_ratio),max_d)
    else:
        new_size = (max_d,int(max_d/aspect_ratio))
    image = cv2.resize(image,new_size)
    return image

def calculate_matches(des1, des2):
    matches = bf.knnMatch(des1, des2, k=2)
    top_results1 = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            top_results1.append([m])

    matches = bf.knnMatch(des2, des1, k=2)
    top_results2 = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            top_results2.append([m])

    top_results = []
    for match1 in top_results1:
        match1_query_index = match1[0].queryIdx
        match1_train_index = match1[0].trainIdx

        for match2 in top_results2:
            match2_query_index = match2[0].queryIdx
            match2_train_index = match2[0].trainIdx

            if (match1_query_index == match2_train_index) and (match1_train_index == match2_query_index):
                top_results.append(match1)
    return top_results

def calculate_score(matches,keypoint1,keypoint2):
    return 100 * (matches/min(keypoint1,keypoint2))

sift = cv2.SIFT_create(10000) # SIFT algorithm with number of keypoints
bf = cv2.BFMatcher() # keypoint matcher

sim_list = [[0 for i in range(img_num)] for j in range(img_num)]
num = len(img_list)
features={} # keypoints and descriptors
tic = time.perf_counter()
for i in range(num):
    img = image_resize(cv2.imread(os.path.join(image_path,img_list[i])))
    keypoints, descriptors = compute_SIFT(img)
    features[i] = (keypoints,descriptors)
for i in range(num):
    keypoints_i, descriptors_i = features[i]
    for j in range(max(0,i-nbrs),min(num,i+nbrs+1)):
        if i>j:
            continue
        keypoints_j, descriptors_j = features[j]
        matches = calculate_matches(descriptors_i, descriptors_j)
        score = calculate_score(len(matches), len(keypoints_i), len(keypoints_j))
        sim_list[i][j] = score
        sim_list[j][i] = sim_list[i][j]
toc = time.perf_counter()
print(f"TIME - {toc-tic:0.2f} s")
for i, line in enumerate(sim_list):
    line = np.divide(line,100)
    line = np.round(line,decimals=3)
    print(line,img_list[i])

TIME - 1.75 s
[1.    0.648 0.628 0.    0.695 0.247] clear.jpg
[0.648 1.    0.539 0.    0.6   0.248] GaussBlur.jpg
[0.628 0.539 1.    0.    0.555 0.202] rotated.jpg
[0.    0.    0.    1.    0.    0.006] inverted.jpg
[0.695 0.6   0.555 0.    1.    0.195] hue_shift.jpg
[0.247 0.248 0.202 0.006 0.195 1.   ] contrast.jpg


From the results we can see that blurred, rotated and hue-shifted images are easily recognized as similar. For image with different contrast the similarity score is lower but still well above our threshold for what is considered similar. The inverted image is however scored as not similar at all which is behaviour that is not preferred. The speed of this algorithm is very good.

TODO - We will try to test those algorithm on bigger number of augmented images to help us reveal if there are any other downsides to said algorithms. For now this is our summary of the results:

Overall these three algorithms have each their upsides and downsides. 
All of them were unable to recognize similarity of inverted picture. This could be solved by calculating every image as normal and inverted version, and we can then take the highest of those scores as our result. This would end up taking about twice as much processing time but for our faster algorithms it might be an effective solution. This requires more testing.

The first implementation of SSIM was the fastest. This came at cost of not very precise results especially for rotated images. Considering its inaccuracy this algorithm isn't ideal for our usage.

The second implementation of DISTS was the slowest. This algorithm was able to recognize similarity in all the testing images other than the inverted one. The higher accuracy is not enough to outweigh the big computing time and so this algorithm is also not usable for us.

The third implementation of SIFT was slower than SSIM but still very reasonable. This algorithm had great accuracy for all images other than inverted one. This algorithm is well-balanced in terms of computing time and accuracy and will likely be the algorithm that we use. The only downside is that for this algorithm to detect inverted images as similar the solution would be harder to implement. TODO - This requires more testing. 